## speedtest
In this notebook we time how long it takes deepstack to process a set of images. The purpose is to allow comparison of speed on different hosts. We also count the total number of predictions as a quality check

```
docker run -e VISION-DETECTION=True -e API-KEY="" -v localstorage:/datastore -p 5000:5000 --name deepstack deepquestai/deepstack:cpu-x4-beta
```

In [1]:
# deepstack credentials
IP_ADDRESS = '192.168.1.84' # 'localhost' # '192.168.1.252'
PORT = 80 # 5000
API_KEY =  "" # if you have not set an api_key, just pass an empty string

In [24]:
import deepstack.core as ds
from pathlib import Path
import time
import sys
import os

In [3]:
images = list(Path("images/").rglob("*.jpg"))

## Object detection

In [4]:
dsobject = ds.DeepstackObject(IP_ADDRESS, PORT, API_KEY)

In [16]:
%%time
total_predictions = 0 # keep predictions
start_time = time.time()

for i, img in enumerate(images):

    try:
        with open(img, 'rb') as image_bytes:
            dsobject.detect(image_bytes)
            predictions = dsobject.predictions
            print(f"Processed image number {i} : {str(img)}, {len(predictions)} predictions")
            total_predictions = total_predictions + len(predictions)
    except Exception as exc:
        print(exc)
        print(img)

end_time = time.time()
duration = end_time - start_time
print(f"Processing completes in {duration}, total of {total_predictions} predictions")

Processed image number 0 : images/scenes/dog.jpg, 1 predictions
Processed image number 1 : images/scenes/street.jpg, 3 predictions
Processed image number 2 : images/scenes/farm.jpg, 9 predictions
Processed image number 3 : images/scenes/pool.jpg, 1 predictions
Processed image number 4 : images/scenes/supermarket.jpg, 0 predictions
Processed image number 5 : images/scenes/crossing.jpg, 11 predictions
Processed image number 6 : images/scenes/porsche.jpg, 1 predictions
Processed image number 7 : images/scenes/people_car.jpg, 5 predictions
Processed image number 8 : images/scenes/test-image3.jpg, 3 predictions
Processed image number 9 : images/faces/me.jpg, 1 predictions
Processed image number 10 : images/faces/idris.jpg, 1 predictions
Processed image number 11 : images/faces/idris_2.jpg, 2 predictions
Processed image number 12 : images/faces/family.jpg, 3 predictions
Processed image number 13 : images/faces/face.jpg, 4 predictions
Processed image number 14 : images/faces/couple.jpg, 3 pre

## League table
| Platform | Speed (sec) | Predictions |
| --- | --- | --- |
| Jetson with jetpack-x3-beta | 6.3 | 48 |
| Jetson with jetpack-x1-beta | 7.5 | 48 |
| Mac Pro with deepstack:cpu-x5-beta | 7.2 | 80 |
| Mac Pro with deepstack:cpu-x4-beta | 6.6 | 80 |
| Mac Pro with deepstack:cpu-x3-beta | 106 | 95 |
| Mac Pro with tflite-server (fastapi) | 2.2 | 178 |
| Mac Pro with tflite-server (flask) | 2.1 | 176 |
| RPi4 with tflite-server (fastapi) | 7.6 | 176 |
| RPi4 with tflite-server (flask) | 8.9 | 159 |

Why fewer predictions with flask?

## Face detection
On Jetson: 

`sudo docker run --runtime nvidia -d -e VISION-FACE=True -e VISION-DETECTION=True -p 80:5000 deepquestai/deepstack:jetpack-x3-beta`

In [11]:
dsface = ds.DeepstackFace(IP_ADDRESS, PORT, API_KEY)

In [29]:
%%time
total_faces = 0 # keep predictions
start_time = time.time()

for i, img in enumerate(images):
    print('----------------------')
    try:
        img_size = os.path.getsize(img)
        img_size = round(img_size/1e6, 2) 
        print(f"{img} : {img_size} MB")
        with open(img, 'rb') as image_bytes:
            dsface.detect(image_bytes)
            predictions = dsface.predictions
            print(f"Processed image number {i} : {str(img)}, {len(predictions)} predictions")
            total_faces = total_faces + len(predictions)
    except Exception as exc:
        print(exc)
        

end_time = time.time()
duration = end_time - start_time
print(f"Processing completes in {duration}, total of {total_faces} faces")

----------------------
images/scenes/dog.jpg : 0.01 MB
Processed image number 0 : images/scenes/dog.jpg, 1 predictions
----------------------
images/scenes/street.jpg : 0.62 MB
Connection error: ('Connection aborted.', BrokenPipeError(32, 'Broken pipe'))
----------------------
images/scenes/farm.jpg : 0.3 MB
Processed image number 2 : images/scenes/farm.jpg, 0 predictions
----------------------
images/scenes/pool.jpg : 0.28 MB
Processed image number 3 : images/scenes/pool.jpg, 0 predictions
----------------------
images/scenes/supermarket.jpg : 1.8 MB
Connection error: ('Connection aborted.', BrokenPipeError(32, 'Broken pipe'))
----------------------
images/scenes/crossing.jpg : 0.11 MB
Processed image number 5 : images/scenes/crossing.jpg, 0 predictions
----------------------
images/scenes/porsche.jpg : 0.12 MB
Processed image number 6 : images/scenes/porsche.jpg, 0 predictions
----------------------
images/scenes/people_car.jpg : 0.11 MB
Processed image number 7 : images/scenes/peopl